In [2]:
import pandas as pd
import numpy as np

# Carregar a base de dados
# No Kaggle, o caminho do arquivo normalmente é '../input/<nome-do-dataset>/<nome-do-arquivo.csv>'
# Assumindo que o arquivo 'global_inflation_data.csv' está na raiz do diretório de input do Kaggle.
file_path = '/kaggle/input/global-inflation-data/global_inflation_data.csv' # Mantenha ou ajuste conforme seu ambiente Kaggle
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    try:
        # Tentar um caminho comum no Kaggle se o arquivo não for encontrado no diretório atual
        # Substitua 'nome-do-seu-dataset' pelo nome da pasta do seu dataset no Kaggle
        # Exemplo: se o seu dataset se chama 'inflation-data', o caminho seria '../input/inflation-data/global_inflation_data.csv'
        df = pd.read_csv('../input/global-inflation-data/global_inflation_data.csv') # Caminho exemplo, ajuste se necessário
    except FileNotFoundError:
        print(f"Erro: Arquivo '{file_path}' não encontrado.")
        print("Por favor, verifique o caminho do arquivo no ambiente Kaggle.")
        print("Ele deve ser algo como '../input/<nome-do-dataset>/global_inflation_data.csv'")
        df = pd.DataFrame() # Cria um DataFrame vazio para evitar mais erros

if not df.empty:
    print("DataFrame original carregado com sucesso.")
    print(f"Shape original: {df.shape}")
    # print(df.head()) # Comentado para não poluir muito a saída inicial
    print("\n" + "="*50 + "\n")

    # 1- Filtrar essa base de dados selecionando apenas as colunas de 2015 até 2024
    # Mantém também 'country_name' e 'indicator_name' para identificação
    colunas_anos = [str(year) for year in range(2015, 2025)] # 2024 inclusive
    colunas_para_manter = ['country_name', 'indicator_name'] + colunas_anos
    
    # Verifica se todas as colunas esperadas existem no DataFrame
    colunas_existentes = [col for col in colunas_para_manter if col in df.columns]
    
    df_filtrado = pd.DataFrame() # Inicializa df_filtrado
    
    if 'country_name' not in colunas_existentes or 'indicator_name' not in colunas_existentes:
        print("Erro: Colunas 'country_name' ou 'indicator_name' não encontradas no DataFrame original.")
        # df_filtrado permanecerá vazio
    else:
        # Filtrar para as colunas existentes
        df_filtrado = df[colunas_existentes].copy() # Usar .copy() para evitar SettingWithCopyWarning

    if df_filtrado.empty:
        print("DataFrame filtrado está vazio (possivelmente devido à falta de 'country_name'/'indicator_name' ou todas as colunas de anos). Não é possível prosseguir.")
    else:
        print("DataFrame filtrado com as colunas de interesse:")
        print(f"Shape após filtragem de colunas: {df_filtrado.shape}")
        print(df_filtrado.head())
        print("\n" + "="*50 + "\n")

        # 2- Apenas informe as linhas que contêm outliers e que contêm dados nulos
        #    NÃO serão removidas.

        # Assegurar que estamos verificando apenas as colunas de anos que de fato existem no df_filtrado
        colunas_anos_no_filtrado = [col for col in colunas_anos if col in df_filtrado.columns]
        
        if not colunas_anos_no_filtrado:
            print("Nenhuma coluna de ano (2015-2024) presente no DataFrame filtrado para verificar nulos ou outliers.")
        else:
            # Criar uma cópia para conversão numérica sem afetar o df_filtrado original para nulos
            df_numerico_para_analise = df_filtrado.copy()
            for col in colunas_anos_no_filtrado:
                df_numerico_para_analise[col] = pd.to_numeric(df_numerico_para_analise[col], errors='coerce')

            # Identificar e informar linhas com dados nulos nas colunas de anos
            # Verifica nulos no df_numerico_para_analise (onde strings não numéricas viraram NaN)
            linhas_com_nulos = df_filtrado[df_numerico_para_analise[colunas_anos_no_filtrado].isnull().any(axis=1)]
            
            if not linhas_com_nulos.empty:
                print(f"IDENTIFICADAS: {len(linhas_com_nulos)} linhas contendo dados nulos (NaN) em pelo menos uma das colunas de 2015-2024:")
                # Para não imprimir todas, pode-se mostrar apenas os identificadores ou uma amostra
                print(linhas_com_nulos[['country_name'] + colunas_anos_no_filtrado].head()) # Mostra as primeiras 5
                # Se quiser ver todas: print(linhas_com_nulos)
            else:
                print("Nenhuma linha com dados nulos encontrada nas colunas de anos (2015-2024).")
            print("\n" + "-"*30 + "\n")

            # Identificar e informar linhas com outliers usando o método do Intervalo Interquartil (IQR)
            # Aplicar apenas às colunas numéricas (anos) que existem e não são totalmente NaN
            # Usar df_numerico_para_analise para os cálculos, já que tem os NaNs corretos
            df_para_outliers_calculo = df_numerico_para_analise[colunas_anos_no_filtrado].dropna(how='all') 

            if not df_para_outliers_calculo.empty and not df_para_outliers_calculo.isnull().all().all(): # Verifica se há dados não-NaN para calcular IQR
                Q1 = df_para_outliers_calculo.quantile(0.25)
                Q3 = df_para_outliers_calculo.quantile(0.75)
                IQR = Q3 - Q1

                # Definir a condição para outliers
                # Aplicar ao df_numerico_para_analise que tem as mesmas dimensões do df_filtrado
                condicao_outlier = (df_numerico_para_analise[colunas_anos_no_filtrado] < (Q1 - 1.5 * IQR)) | \
                                   (df_numerico_para_analise[colunas_anos_no_filtrado] > (Q3 + 1.5 * IQR))
                
                # Filtrar as linhas originais (do df_filtrado) que contêm pelo menos um outlier
                linhas_com_outliers = df_filtrado[condicao_outlier.any(axis=1)]
                
                if not linhas_com_outliers.empty:
                    print(f"IDENTIFICADAS: {len(linhas_com_outliers)} linhas contendo outliers (IQR) em pelo menos uma das colunas de 2015-2024:")
                    # Para não imprimir todas, pode-se mostrar apenas os identificadores ou uma amostra
                    print(linhas_com_outliers[['country_name'] + colunas_anos_no_filtrado].head()) # Mostra as primeiras 5
                    # Se quiser ver todas: print(linhas_com_outliers)
                else:
                    print("Nenhuma linha com outliers (IQR) identificada nas colunas de anos (2015-2024).")
            else:
                print("Não foi possível calcular outliers (IQR) pois não há dados suficientes ou todos os valores são NaN após a coerção numérica nas colunas de anos.")
        
        print("\n" + "="*50 + "\n")
        print("O DataFrame abaixo ('df_filtrado') contém todas as linhas originais,")
        print("apenas com as colunas selecionadas. As linhas com nulos/outliers foram APONTADAS acima, não removidas.")
        print(f"Shape do DataFrame final (apenas colunas filtradas): {df_filtrado.shape}")
        print(df_filtrado.head())
        print("\n" + "="*50 + "\n")

        # Salvar o DataFrame filtrado (com todas as linhas, apenas colunas selecionadas) em um novo arquivo CSV
        nome_arquivo_saida = 'dados_inflacao_colunas_filtradas_2015_2024.csv'
        try:
            df_filtrado.to_csv(nome_arquivo_saida, index=False)
            print(f"Arquivo com colunas filtradas (sem remoção de linhas por nulos/outliers) salvo com sucesso como '{nome_arquivo_saida}'")
        except Exception as e:
            print(f"Erro ao salvar o arquivo CSV: {e}")
else:
    print("DataFrame não foi carregado. Não é possível executar as operações.")



DataFrame original carregado com sucesso.
Shape original: (196, 47)


DataFrame filtrado com as colunas de interesse:
Shape após filtragem de colunas: (196, 12)
  country_name                                   indicator_name  2015   2016  \
0  Afghanistan  Annual average inflation (consumer prices) rate -0.66   4.38   
1      Albania  Annual average inflation (consumer prices) rate  1.90   1.30   
2      Algeria  Annual average inflation (consumer prices) rate  4.80   6.40   
3      Andorra  Annual average inflation (consumer prices) rate -1.10  -0.40   
4       Angola  Annual average inflation (consumer prices) rate  9.20  30.70   

    2017   2018  2019   2020   2021   2022  2023  2024  
0   4.98   0.63   2.3   5.44   5.06  13.71   9.1   NaN  
1   2.00   2.00   1.4   1.60   2.00   6.70   4.8   4.0  
2   5.60   4.30   2.0   2.40   7.20   9.30   9.0   6.8  
3   2.60   1.00   0.5   0.10   1.70   6.20   5.2   3.5  
4  29.80  19.60  17.1  22.30  25.80  21.40  13.1  22.3  


IDENTIFICADAS: